In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, SpatialDropout1D, Activation, concatenate
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers.advanced_activations import ReLU, PReLU, LeakyReLU, ELU
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from tensorflow.keras.utils import plot_model

데이터 읽어오기

In [ ]:
enrolled = pd.read_csv("enrolleds_210729.csv")
#hw = pd.read_csv("./sparta_csv/enrolleds_homeworks_210729.csv")
user_data = pd.read_csv("prequestions_210729.csv", encoding='cp949')
#rounds = pd.read_csv("./sparta_csv/rounds_210729.csv", encoding='cp949')
course = pd.read_csv("courses_210729.csv", encoding='cp949')

print(user_data.shape)
print(enrolled.shape)
print(course.shape)

(48708, 6)
(32154, 7)
(50, 2)


데이터 모양 확인하기

In [ ]:
user_data.describe()

,age
count,4.870800e+04
mean,3.757061e+04
std,5.386966e+05
min,0.000000e+00
25%,2.400000e+01
50%,2.800000e+01
75%,3.400000e+01
max,2.009043e+07


In [ ]:
enrolled.describe()

,is_registered
count,32153.000000
mean,0.925979
std,0.261809
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
course.describe()

,course_id,title
count,50,50
unique,50,50
top,bdd218f17ec6313b96f3,웹개발 종합반
freq,1,1


# **04. 데이터 전처리하기**

유저데이터 전처리

In [ ]:
####유저데이터 확인
user_data.head()

,user_id,course_id,round_id,job,age,gender
0,2397731931fe978eb349,bdd218f17ec6313b96f3,655f288f4382b12fd3d9,직장인,30,남
1,5eb42ddf1cda5ff826f2,bdd218f17ec6313b96f3,655f288f4382b12fd3d9,창업/사업,34,남
2,012a9585146a4ddc9f29,bdd218f17ec6313b96f3,655f288f4382b12fd3d9,직장인,32,여
3,347cba891997ca61b13e,cf06e86a19be34ec1e78,736c6b3d15fe95f501f8,창업/사업,34,여
4,caa3368273ce3b330b3e,cf06e86a19be34ec1e78,736c6b3d15fe95f501f8,대학생,23,남


Job 유형 분석
:Job이 지저분하기 때문에 이를 분류화함

In [ ]:
##job 유형 분석
user_data.job.unique()

array(['직장인', '창업/사업', '대학생', '무직/준비중', '기타', '학생', '2. 직장인', '4. 무직/준비중',
       '1. 학생', '5. 기타', '3. 창업/사업'], dtype=object)

**데이터 정제하기**

In [ ]:
###포맷 맞추기
user_data = user_data[['user_id','job','age','gender']]
user_data['job'] = user_data['job'].replace({'2. 직장인':'직장인', '4. 무직/준비중':'무직/준비중', '1. 학생':'학생',
                                            '5. 기타':'기타','3. 창업/사업' : '창업/사업'})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


**Job 정제결과 확인하기**

In [ ]:
##job 정제 결과 확인하기
user_data.job.unique()

array(['직장인', '창업/사업', '대학생', '무직/준비중', '기타', '학생'], dtype=object)

**불필요하거나 유효하지 않은 데이터 삭제하기**

In [ ]:
enroll_data = enrolled[['course_id', 'user_id', 'is_registered']]
enroll_data.describe()
user_data.describe()

#키값 중복제거 (첫번째만 살려놓겠다)
user_data=user_data.drop_duplicates(subset='user_id', keep='first', inplace=False)
#100세이상 제거 
user_data=user_data[user_data['age']<100]

**정제결과 확인**

In [ ]:
## 중복제거 후 사용자(user_id) 수는?
print(len(user_data.user_id.unique()))
## 상위 5개 행 좀 볼까?
user_data.head()

40039


,user_id,job,age,gender
0,2397731931fe978eb349,직장인,30,남
1,5eb42ddf1cda5ff826f2,창업/사업,34,남
2,012a9585146a4ddc9f29,직장인,32,여
3,347cba891997ca61b13e,창업/사업,34,여
4,caa3368273ce3b330b3e,대학생,23,남


In [ ]:
## 데이터구조 한번 볼까나?
user_data.shape

(40039, 4)

In [ ]:
## 데이터 구성 다시 한번 살펴 볼까?
user_data.describe()

,age
count,40039.000000
mean,29.273259
std,8.372794
min,0.000000
25%,24.000000
50%,28.000000
75%,33.000000
max,99.000000


# **enroll 데이터 전처리**

In [ ]:
## Enroll 데이터 확인
enrolled.head()
enroll_data = enrolled[['course_id', 'user_id', 'is_registered']]
enroll_data.head()

,course_id,user_id,is_registered
0,bdd218f17ec6313b96f3,2151609483dab408339c,1.0
1,bdd218f17ec6313b96f3,aaf0bbc09859eb9d2c51,0.0
2,bdd218f17ec6313b96f3,aaf0bbc09859eb9d2c51,0.0
3,bdd218f17ec6313b96f3,dbe1e66d55175b5f7ff0,0.0
4,bdd218f17ec6313b96f3,dbe1e66d55175b5f7ff0,0.0


In [ ]:
## enroll_data 갯수 확인해 볼끼
enroll_data.shape

(32154, 3)

In [ ]:
## enrolled_data 데이터 구조 살펴볼까?
enroll_data.describe()

,is_registered
count,32153.000000
mean,0.925979
std,0.261809
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
## 유저데이터가 있는 interaction 데이터만 추출
enroll_data = enroll_data[enroll_data["user_id"].isin(list(user_data.user_id.unique()))]

In [ ]:
## 결과 한번 볼까?
enroll_data.shape
## 67,321개 -> 53,226개로 축소

(27449, 3)

In [ ]:
##중복된 데이터 제거하기
## 전체 레코드가 각각 Unique해야 돼!
enroll_data=enroll_data.drop_duplicates(subset=None, keep='first', inplace=False)

In [ ]:
## 중복제거 결과 확인하기
enroll_data.shape
## 53,2226개 --> 52,111개로 축소

(26599, 3)

In [ ]:
## 5개 행 자료 다시 볼까?
enroll_data.head()

,course_id,user_id,is_registered
1,bdd218f17ec6313b96f3,aaf0bbc09859eb9d2c51,0.0
3,bdd218f17ec6313b96f3,dbe1e66d55175b5f7ff0,0.0
7,bdd218f17ec6313b96f3,80c1857389ec2d56928f,1.0
10,bdd218f17ec6313b96f3,a9899ce51cdd8a6264af,1.0
13,bdd218f17ec6313b96f3,ba56f3de9b03501c6651,1.0


In [ ]:
## enroll_data 전체 데이터 살펴볼까?
enroll_data.describe

<bound method NDFrame.describe of                   course_id               user_id  is_registered
1      bdd218f17ec6313b96f3  aaf0bbc09859eb9d2c51            0.0
3      bdd218f17ec6313b96f3  dbe1e66d55175b5f7ff0            0.0
7      bdd218f17ec6313b96f3  80c1857389ec2d56928f            1.0
10     bdd218f17ec6313b96f3  a9899ce51cdd8a6264af            1.0
13     bdd218f17ec6313b96f3  ba56f3de9b03501c6651            1.0
...                     ...                   ...            ...
32145  1de9424b817aff292c6b  bc00db6236d8c75d967f            1.0
32148  1de9424b817aff292c6b  68c1401469df7cd09278            1.0
32149  1de9424b817aff292c6b  9613a7af4b935a9c740a            1.0
32151  1de9424b817aff292c6b  df0ccfa4a27a9a79698a            1.0
32152  1de9424b817aff292c6b  527b10f703ff5f42d7aa            1.0

[26599 rows x 3 columns]>

In [ ]:
## enroll_data 데이터 구조는?
enroll_data.describe()

,is_registered
count,26599.000000
mean,0.951427
std,0.214978
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


# **Course 데이터 전처리하기**

In [ ]:
## course 데이터 확인
course.head()

,course_id,title
0,bdd218f17ec6313b96f3,웹개발 종합반
1,cf06e86a19be34ec1e78,앱개발 종합반
2,8e02dc4b2b82e9a15e1e,파이썬 데이터분석 첫걸음
3,d08d0754e8d4eec3862b,이미지처리로 시작하는 딥러닝
4,c6de4714a96cce9afe82,엑셀보다 쉬운 SQL


title이 너무 Real하여 Deep Learning하기에는 부적합!
따라서 Excel 노가다(^^)작업을 통해 데이터 재구성 후 다시 업로드

In [ ]:
## 파일 업로드 후 다시 읽어 들이기!
course2 = pd.read_csv("courses_210729_v2.csv", encoding='cp949')

In [ ]:
##데이터 들여다보기
course2.head()

,course_id,app,level
0,bdd218f17ec6313b96f3,웹,종합
1,cf06e86a19be34ec1e78,앱,종합
2,8e02dc4b2b82e9a15e1e,파이썬,초급
3,d08d0754e8d4eec3862b,파이썬,종합
4,c6de4714a96cce9afe82,SQL,초급


# **05. Input Data로 변환**

**없는 데이터 만들어내기**

왜냐? enroll_data를 봤더니
```
	is_registered
count	52111.000000
mean	0.966571
std	0.179755
min	0.000000
25%	1.000000
50%	1.000000
75%	1.000000
max	1.000000

```
평균이 0.96이다. 거의 1임을 알 수 있다.
그래서, 0과 1을 섞은 데이터를 임의로 만들어 내려고 하는 거다. 오직 실습을 위한 통계분석을 위해서



In [ ]:
## 엄청 오래걸림에 유의할 것!
## 런타임유형: 하드웨어 가속기:None: 7분36초 
## 런타임유형: 하드웨어 가속기:GPU:  5분19초. 근데 Google에서 "GPU를 활용하고 있지 않습니다. 표준런타임으로 변경하시겠습니까? 라는 메시지가 나온다. TPU로 다시 테스트해 본다"
## 런타임유형: 하드웨어 가속기:TPU: 6분 28최. 단, 데이터가 줄어들었는데도 시간은 더 걸린다? 일반 데이터 처리용으로는 가장 부적합함.
## 결국, GPU로 설정 후 전체 재수행함!!
## 런타임유형: 하드웨어 가속기:GPU:  5분19초. 근데 data가 줄었는데도 시간은 그대로다?

import random 
unique_user = list(user_data.user_id.unique())
unique_course = list(enroll_data.course_id.unique())

user_list = random.choices(unique_user,  k=30000)
course_list = random.choices(unique_course,  k=30000)

print(enroll_data.shape)

for i in range(len(user_list)):
  user = user_list[i]
  course = course_list[i]
  try:
    value = int(enroll_data.loc[(enroll_data["course_id"] ==course) & (enroll_data["user_id"] == user),'is_registered'])
  except TypeError:
    enroll_data = enroll_data.append({'course_id':course, 'user_id':user, 'is_registered':0}, ignore_index=True)    
print(enroll_data.shape)

(26599, 3)
(54191, 3)


In [ ]:
## enroll_data 데이터 확인
enroll_data.head()

,course_id,user_id,is_registered
0,bdd218f17ec6313b96f3,aaf0bbc09859eb9d2c51,0.0
1,bdd218f17ec6313b96f3,dbe1e66d55175b5f7ff0,0.0
2,bdd218f17ec6313b96f3,80c1857389ec2d56928f,1.0
3,bdd218f17ec6313b96f3,a9899ce51cdd8a6264af,1.0
4,bdd218f17ec6313b96f3,ba56f3de9b03501c6651,1.0


In [ ]:
## enroll_data 데이터 구조 확인
enroll_data.shape

(54191, 3)

**이해를 쉽게 하기 위해 enroll_data에 User정보 칼럼 추가!**

In [ ]:
enroll_data = enroll_data.merge(user_data, left_on = 'user_id', right_on='user_id', how = 'left')

In [ ]:
## 데이터를 다시 확인
user_data.head()

,user_id,job,age,gender
0,2397731931fe978eb349,직장인,30,남
1,5eb42ddf1cda5ff826f2,창업/사업,34,남
2,012a9585146a4ddc9f29,직장인,32,여
3,347cba891997ca61b13e,창업/사업,34,여
4,caa3368273ce3b330b3e,대학생,23,남


In [ ]:
##enroll_data 다시 확인
enroll_data.head()

,course_id,user_id,is_registered,job,age,gender
0,bdd218f17ec6313b96f3,aaf0bbc09859eb9d2c51,0.0,직장인,29,남
1,bdd218f17ec6313b96f3,dbe1e66d55175b5f7ff0,0.0,창업/사업,29,여
2,bdd218f17ec6313b96f3,80c1857389ec2d56928f,1.0,직장인,30,여
3,bdd218f17ec6313b96f3,a9899ce51cdd8a6264af,1.0,무직/준비중,24,여
4,bdd218f17ec6313b96f3,ba56f3de9b03501c6651,1.0,무직/준비중,24,여


In [ ]:
## 데이터 갯수 재확인 (행은 그대로, 컬럼만 3개 늘어났다. OK )
enroll_data.shape

(60306, 6)

# **Course2 데이터 처리**

In [1]:
## Course2 데이터 확인하기
course2.head()

NameError: ignored

**이해를 쉽게 하기 위해 enroll_data에 Course 정보 칼럼 추가!**

In [ ]:
enroll_data = enroll_data.merge(course2, left_on = 'course_id', right_on='course_id', how = 'left')

In [ ]:
##enroll_data 다시 확인
enroll_data.head()

In [ ]:
## 데이터 갯수 재확인 (행은 그대로, 컬럼만 2개 더 늘어났다(6->8개). OK )
enroll_data.shape

In [ ]:
## 데이터 구조 확인. is_registered 평균은 0.52, Age 평균은 29.3세임을 알 수 있다!
enroll_data.describe()

# **Y값을 분리. 왜?**
Y값은 0과1 즉, 들을 건지 안들을 건지를 추천해 주어야 하니까. 이게 중요하니까

In [ ]:
### Y값(label) 분리
y = enroll_data['is_registered'].values
## enroll_data에서는 is_registered 컬럼을 삭제
enroll_data.drop('is_registered', axis = 1, inplace=True)

In [ ]:
## y 값 살펴보기
print(y)
##y값 갯수 = enroll_data의 갯수 +> OK
print(len(y))

[0. 0. 1. ... 0. 0. 0.]
54191


In [ ]:
##enroll_data 다시 확인
enroll_data.head()

,course_id,user_id,job,age,gender,app,level
0,bdd218f17ec6313b96f3,aaf0bbc09859eb9d2c51,직장인,29,남,웹,종합
1,bdd218f17ec6313b96f3,dbe1e66d55175b5f7ff0,창업/사업,29,여,웹,종합
2,bdd218f17ec6313b96f3,80c1857389ec2d56928f,직장인,30,여,웹,종합
3,bdd218f17ec6313b96f3,a9899ce51cdd8a6264af,무직/준비중,24,여,웹,종합
4,bdd218f17ec6313b96f3,ba56f3de9b03501c6651,무직/준비중,24,여,웹,종합


# **분석을 위한 카테고리 텍스트 데이터 변환**

In [ ]:
#카테고리를 숫자로 
CATEGORICAL_COLUMNS = ["job", "gender", "app", "level"]
CONTINUOUS_COLUMNS = ['age']
for c in CATEGORICAL_COLUMNS:
    le = LabelEncoder()
    enroll_data[c] = le.fit_transform(enroll_data[c])

In [ ]:
##enroll_data 다시 확인
enroll_data.head()

,course_id,user_id,job,age,gender,app,level
0,bdd218f17ec6313b96f3,aaf0bbc09859eb9d2c51,3,29,0,3,0
1,bdd218f17ec6313b96f3,dbe1e66d55175b5f7ff0,4,29,1,3,0
2,bdd218f17ec6313b96f3,80c1857389ec2d56928f,3,30,1,3,0
3,bdd218f17ec6313b96f3,a9899ce51cdd8a6264af,2,24,1,3,0
4,bdd218f17ec6313b96f3,ba56f3de9b03501c6651,2,24,1,3,0


In [ ]:
print(enroll_data)

                  course_id               user_id  job  age  gender  app  \
0      bdd218f17ec6313b96f3  aaf0bbc09859eb9d2c51    3   29       0    3   
1      bdd218f17ec6313b96f3  dbe1e66d55175b5f7ff0    4   29       1    3   
2      bdd218f17ec6313b96f3  80c1857389ec2d56928f    3   30       1    3   
3      bdd218f17ec6313b96f3  a9899ce51cdd8a6264af    2   24       1    3   
4      bdd218f17ec6313b96f3  ba56f3de9b03501c6651    2   24       1    3   
...                     ...                   ...  ...  ...     ...  ...   
54186  ed94b102f985c7187ff4  7c7e96cc286f968352cf    3   37       0    5   
54187  7d9b4bfbdee3dadaa249  40de042e98147ef33e1a    3   29       1    1   
54188  bdd218f17ec6313b96f3  e200b585882ea6b10931    3   31       1    3   
54189  0bbb55030fc7171716ff  a7dca85dd5a778334a77    1   24       0    3   
54190  c6de4714a96cce9afe82  4b3085a7c60303ca6c78    3   38       1    0   

       level  
0          0  
1          0  
2          0  
3          0  
4          0

# **학습, 테스트 데이터 나누어주기**

In [ ]:
print(y)

[0. 0. 1. ... 0. 0. 0.]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(enroll_data, y, test_size=0.3, random_state=42)

In [ ]:
X_train_category = np.array(X_train[CATEGORICAL_COLUMNS])
X_test_category = np.array(X_test[CATEGORICAL_COLUMNS])
X_train_countinue = np.array(X_train[CONTINUOUS_COLUMNS], dtype='float64')
X_test_countinue = np.array(X_test[CONTINUOUS_COLUMNS], dtype='float64')

In [ ]:
print(X_train_category[:1])
print(X_test_category[:1])
print(X_train_countinue[:1])
print(X_test_countinue[:1])


[[2 1 5 2]]
[[1 1 5 2]]
[[25.]]
[[22.]]


In [ ]:
#####정규화

scaler = StandardScaler()
X_train_countinue = scaler.fit_transform(X_train_countinue)
X_test_countinue = scaler.transform(X_test_countinue)

In [ ]:
## 정규화 결과 확인
print(X_train_countinue[:1])
print(X_test_countinue[:1])

[[-0.55276103]]
[[-0.92759583]]


# **Polynomial하게 바꾸기**

In [ ]:
###########Polynomial 하게 바꿔줌 (비선형적인 설정으로 선형 회귀를 확장하는 방법. 즉 다항식 함수로 바꿔줌)

poly = PolynomialFeatures(degree=2, interaction_only=True)
X_train_category_poly = poly.fit_transform(X_train_category)
X_test_category_poly = poly.transform(X_test_category)

In [ ]:
## 결과 확인
print(X_train_category_poly[:1])
print(X_test_category_poly[:1])

[[ 1.  2.  1.  5.  2.  2. 10.  4.  5.  2. 10.]]
[[ 1.  1.  1.  5.  2.  1.  5.  2.  5.  2. 10.]]


In [ ]:
## 비선형 array값들을 선형형태로 바꾸어줌. PolynomialFeatures 함수를 쓰면 쉽게 바꿀 수 있음
n = np.array([[1,2,3]])
poly2 = PolynomialFeatures(degree = 2, interaction_only = True)
print(poly.fit_transform(n))

[[1. 1. 2. 3. 2. 3. 6.]]


# **06. 모델 설계하기**

**심플한 Wide & Deep 모델 설계**

**Deep 모델 설계하기**

In [ ]:
def get_deep_model():

    category_inputs = []
    category_embeds = []
    # 카테고리컬 데이터 임베딩
    for i in range(len(CATEGORICAL_COLUMNS)):
        input_i = Input(shape=(1,), dtype='int32')
        dim = len(np.unique(enroll_data[CATEGORICAL_COLUMNS[i]]))
        embed_dim = int(np.ceil(dim ** 0.5)) # embedding 차원을 0.5배 정도로 해서 한다.
        embed_i = Embedding(dim, embed_dim, input_length=1)(input_i)
        flatten_i = Flatten()(embed_i)
        category_inputs.append(input_i)
        category_embeds.append(flatten_i)
    # continuous 데이터 input
    continue_input = Input(shape=(len(CONTINUOUS_COLUMNS),))
    continue_dense = Dense(256, use_bias=False)(continue_input)
    # category와 continue를 합침
    concat_embeds = concatenate([continue_dense] + category_embeds)
    concat_embeds = Activation('relu')(concat_embeds)
    bn_concat = BatchNormalization()(concat_embeds)

    fc1 = Dense(512, use_bias=False)(bn_concat)
    relu1 = ReLU()(fc1)
    bn1 = BatchNormalization()(relu1)
    fc2 = Dense(256, use_bias=False)(bn1)
    relu2 = ReLU()(fc2)
    bn2 = BatchNormalization()(relu2)
    fc3 = Dense(128)(bn2)
    relu3 = ReLU()(fc3)
    
    return category_inputs, continue_input, relu3

**Wide모델 설계**

In [ ]:
def get_wide_model():
    dim = X_train_category_poly.shape[1]
    return Input(shape=(dim,))

# **Deep & Wide 모델 합치기**

In [ ]:
category_inputs, continue_input, deep_model = get_deep_model()
wide_model = get_wide_model()

In [ ]:
####################WIDE 랑 DEEP이랑 합치기############################
out_layer = concatenate([deep_model, wide_model])
inputs = [continue_input] + category_inputs + [wide_model]
output = Dense(1, activation='sigmoid')(out_layer)
model = Model(inputs=inputs, outputs=output)

# **모델 확인하기**

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                              

# **07. 모델 학습하고 결과를 보여주기**

**데이터 입력하고 학습하기 **

In [ ]:
################입력데이터###########################

input_data = [X_train_countinue] + [X_train_category[:, i] for i in range(X_train_category.shape[1])] + [X_train_category_poly]


epochs = 30
optimizer ='adam'
batch_size = 128

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
## callback 구문 에러가 나서 빼버리고 처리함. 근데 돌아감.
#model.fit(input_data, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.15, callbacks=[checkpoint, early_stopping])
model.fit(input_data, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.15)

Epoch 1/30
252/252 [==============================] - 7s 9ms/step - loss: 0.5990 - accuracy: 0.7067 - val_loss: 0.7808 - val_accuracy: 0.5225
Epoch 2/30
252/252 [==============================] - 2s 8ms/step - loss: 0.5379 - accuracy: 0.7486 - val_loss: 0.5656 - val_accuracy: 0.7285
Epoch 3/30
252/252 [==============================] - 2s 8ms/step - loss: 0.5268 - accuracy: 0.7546 - val_loss: 0.5078 - val_accuracy: 0.7692
Epoch 4/30
252/252 [==============================] - 2s 8ms/step - loss: 0.5192 - accuracy: 0.7566 - val_loss: 0.5218 - val_accuracy: 0.7649
Epoch 5/30
252/252 [==============================] - 2s 8ms/step - loss: 0.5160 - accuracy: 0.7579 - val_loss: 0.5067 - val_accuracy: 0.7624
Epoch 6/30
252/252 [==============================] - 2s 8ms/step - loss: 0.5099 - accuracy: 0.7596 - val_loss: 0.5033 - val_accuracy: 0.7712
Epoch 7/30
252/252 [==============================] - 2s 9ms/step - loss: 0.5110 - accuracy: 0.7597 - val_loss: 0.5009 - val_accuracy: 0.7701
Epoch 

**결과 확인. 정확도(test_acc) 확인**

In [ ]:
eval_input_data = [X_test_countinue] + [X_test_category[:, i] for i in range(X_test_category.shape[1])] + [X_test_category_poly]
loss, acc = model.evaluate(eval_input_data, y_test)
print(f'test_loss:{loss} - test_acc:{acc}')

509/509 [==============================] - 2s 3ms/step - loss: 0.5155 - accuracy: 0.7569
test_loss:0.515516996383667 - test_acc:0.7568581700325012


In [ ]:
y_test.sum()

7586.0